# Домашнее задание № 3. Исправление опечаток

In [1]:
%%capture
!pip install textdistance

In [2]:
import re
import textdistance

from collections import Counter, defaultdict
from itertools import combinations
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from string import punctuation
from tqdm.notebook import tqdm

In [3]:
punctuation += "«»—…“”"

In [4]:
bad = open("data/sents_with_mistakes.txt", encoding="utf8").read().splitlines()
true = open("data/correct_sents.txt", encoding="utf8").read().splitlines()

In [5]:
corpus = open("data/wiki_data.txt", encoding="utf8").read()
vocab = Counter(re.findall("\w+", corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(
    analyzer="char",
    max_features=10000,
    ngram_range=(1, 3)
)
X = vec.fit_transform(vocab)

In [6]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

In [7]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    distances = cosine_distances(v, X)[0]
    topn = distances.argsort()[:topn]

    # top n with smallest distances
    return [(id2word[top], distances[top]) for top in topn]

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [8]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [9]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    similarities = Counter()
    for word in lookup:
        similarities[word] = (
            metric.normalized_similarity(text, word),
            P(word, N=N)
        )
    closest = similarities.most_common(topn)
    return closest

In [10]:
get_closest_hybrid_match("сонце", X, vec, topn=10)

[('солнце', (0.8333333333333334, 2.4440966240624417e-05)),
 ('конце', (0.8, 0.00037068798798280367)),
 ('соне', (0.8, 1.745783302901744e-06)),
 ('монце', (0.8, 1.5518073803571057e-06)),
 ('донце', (0.8, 3.879518450892764e-07)),
 ('солнцем', (0.7142857142857143, 8.340964669419444e-06)),
 ('солнцев', (0.7142857142857143, 7.759036901785528e-07)),
 ('олонце', (0.6666666666666667, 3.879518450892764e-07)),
 ('ньонце', (0.6666666666666667, 1.939759225446382e-07)),
 ('донцем', (0.6666666666666667, 1.939759225446382e-07))]

In [11]:
def evaluate_spellcheck(true, bad, fn, fn_kwargs={}):
    mistakes = []
    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    total = 0
    correct = 0

    cashed = {}
    for i in tqdm(range(len(true))):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            if predict_mistaken(pair[1], vocab):
                pred = cashed.get(pair[1], fn(pair[1], **fn_kwargs)[0][0])
                cashed[pair[1]] = pred
            else:
                pred = pair[1]


            if pred == pair[0]:
                correct += 1
            else:
                mistakes.append((pair[0], pair[1], pred))
            total += 1

            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] != pred:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == pred:
                    mistaken_fixed += 1

    print("correct/total:", correct/total)
    print("mistaken_fixed/total_mistaken:", mistaken_fixed/total_mistaken)
    print("correct_broken/total_correct:", correct_broken/total_correct)

In [12]:
fn_kwargs = {"X": X, "vec": vec}
evaluate_spellcheck(true, bad, get_closest_hybrid_match, fn_kwargs)

  0%|          | 0/915 [00:00<?, ?it/s]

correct/total: 0.856128064032016
mistaken_fixed/total_mistaken: 0.4922360248447205
correct_broken/total_correct: 0.09004249454461927


## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов

2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!)

3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления

4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [13]:
class Symspell:
    def __init__(self, vocab, max_deletions=1):
        self.vocab = vocab
        self.total_words = sum(vocab.values())
        self.max_deletions = max_deletions
        self.deletion_dict = self.get_deletion_dict(vocab)

    def get_proba(self, word):
        return self.vocab[word] / self.total_words

    def get_deletions(self, word):
        deletions = []
        min_len = max(1, len(word) - self.max_deletions)
        for i in range(min_len, len(word)):
            deletions.extend(
                ["".join(x) for x in combinations(word, i)]
            )
        return set(deletions)

    def get_deletion_dict(self, vocab):
        deletion_dict = defaultdict(lambda: [])
        for word in vocab:
            deletions = self.get_deletions(word)
            for deletion in deletions:
                deletion_dict[deletion].append(word)
        return deletion_dict

    def get_closest_match(self, word, topn=10):
        candidate_probas = Counter()
        deletions_for_word = self.get_deletions(word)
        # add the misspelled word itself
        deletions_for_word.add(word)
        for deletion in deletions_for_word:
            if deletion in self.deletion_dict:
                correct_words = self.deletion_dict[deletion]
                for correct_word in correct_words:
                    candidate_probas[correct_word] = self.get_proba(correct_word)
        closest = candidate_probas.most_common(topn)
        return closest if len(closest) > 0 else word

In [14]:
symspell = Symspell(vocab, max_deletions=1)
symspell.get_closest_match("сонце", topn=10)

[('конце', 0.00037068798798280367),
 ('солнце', 2.4440966240624417e-05),
 ('сотне', 4.267470295982041e-06),
 ('монце', 1.5518073803571057e-06),
 ('соней', 1.1638555352678294e-06),
 ('сонет', 9.698796127231912e-07),
 ('сконе', 7.759036901785528e-07),
 ('сионе', 5.819277676339147e-07),
 ('сосне', 3.879518450892764e-07),
 ('согне', 3.879518450892764e-07)]

In [15]:
evaluate_spellcheck(true, bad, symspell.get_closest_match)

  0%|          | 0/915 [00:00<?, ?it/s]

correct/total: 0.8326163081540771
mistaken_fixed/total_mistaken: 0.30978260869565216
correct_broken/total_correct: 0.09004249454461927
